In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(stoi)
print(vocab_size)
print(itos)

27
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [7]:
block_size = 3

# building the dataset
def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '->', itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [13]:
n_embed = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g)  # 27 chars, embedding dim 2

# Layer 1
W1 = torch.randn((n_embed * block_size, n_hidden), generator=g) * (5/3)/((n_embed * block_size) ** 0.5)  # 6 chars, 100 hidden units
b1 = torch.randn(n_hidden, generator=g)  * 0.1

# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g)  * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

# Batchnorm params
bngain = torch.ones((1, n_hidden))* 0.1 + 1.0  # 100 hidden units
bnbias = torch.zeros((1, n_hidden))* 0.1  # 100 hidden units

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print('Number of parameters:', sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

Number of parameters: 4137


In [55]:
# lri = []
# lossi = []
for i in range(50000):
    # construct a mini-batch of data
    ix = torch.randint(0, Xtr.shape[0], (32,))  # 32 random indices
    # forward pass
    emb = C[Xtr[ix]]
    h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Ytr[ix])

    # backward pass
    for p in parameters:
        p.grad = None  # zero the gradients

    # compute gradients
    loss.backward()

    #lr = lrs[i]
    lr = 0.01
    for p in parameters:
        p.data -= lr * p.grad  # gradient descent step
    # lri.append(lr)
    # lossi.append(loss.log10().item())
print(loss.item())

2.5092339515686035


In [56]:
emb = C[Xtr]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ytr)
loss.item()

2.169128894805908

In [57]:
emb = C[Xdev]
h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Ydev)
loss.item()

2.202073335647583

In [58]:
# sample from the model
g = torch.Generator().manual_seed(2147483647+ 10)
for i in range(20):
    out = []
    context = [0] * 3
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(-1, 30) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:] + [ix]
        out.append(ix)
        if ix == 0:
            break
    print(''.join(itos[i] for i in out))

mona.
kayanniellend.
ryla.
rethan.
endraeg.
aderedielin.
shi.
jenre.
elieananarleitzion.
kamin.
shdyno.
shiriel.
kindreellerxn.
profour.
kavear.
yarleyeh.
yuma.
maskeyla.
halina.
yansun.
